In [5]:
import imaplib
import email
import keyring
import snowflake.connector
from email.header import decode_header
import datetime
import configparser

In [6]:
# Gmail credentials
USERNAME = 'ananyamehta.work@gmail.com'
APP_PASSWORD = keyring.get_password("my_app", USERNAME)
if not APP_PASSWORD:
    raise ValueError("App password not found in keyring. Please store it first.")

In [7]:
# Snowflake connection details
config = configparser.ConfigParser()
config.read('config.ini')

SNOWFLAKE_USER = config['snowflake']['user']
SNOWFLAKE_PASSWORD = config['snowflake']['password']
SNOWFLAKE_ACCOUNT = config['snowflake']['account']
SNOWFLAKE_WAREHOUSE = config['snowflake']['warehouse']
SNOWFLAKE_DATABASE = config['snowflake']['database']
SNOWFLAKE_SCHEMA = config['snowflake']['schema']

In [8]:
# Initialize Snowflake connection and cursor
conn = None
cur = None
mail = None

try:
    # Connecting to Snowflake
    conn = snowflake.connector.connect(
        user=SNOWFLAKE_USER,
        password=SNOWFLAKE_PASSWORD,
        account=SNOWFLAKE_ACCOUNT,
        warehouse=SNOWFLAKE_WAREHOUSE,
        database=SNOWFLAKE_DATABASE,
        schema=SNOWFLAKE_SCHEMA
    )
    cur = conn.cursor()

    # Initializing database
    cur.execute(f"USE DATABASE {SNOWFLAKE_DATABASE};")
    cur.execute(f"USE SCHEMA {SNOWFLAKE_SCHEMA};")

    # Creating table if it doesn't exist
    cur.execute("""
        CREATE TABLE IF NOT EXISTS email_alertdata (
            subject STRING,
            INCIDENT_ID STRING,
            impact_level STRING,
            affected_service STRING,
            timestamp TIMESTAMP,
            description STRING
        );
    """)

    # Connecting to Gmail IMAP
    mail = imaplib.IMAP4_SSL('imap.gmail.com')
    mail.login(USERNAME, APP_PASSWORD)
    mail.select('inbox')

    # Get today's date in the correct IMAP format (DD-MMM-YYYY)
    today = datetime.datetime.today().strftime("%d-%b-%Y")

    # Search for alert emails
    status, messages = mail.search(None, f'(SINCE "{today}" SUBJECT "alert")')
    email_ids = messages[0].split()

    if not email_ids:
        print(f"No alert emails found today ({today}).")
        email_count = 0  # No emails found
    else:
        email_count = len(email_ids)
        print(f"Found {email_count} alert emails")

    # Process emails and extract details
    print(f"Fetching {email_count} alert emails received today ({today})...")
    success_count = 0

    # Process emails only if there are emails to process
    if email_count > 0:
        for recent_email_id in email_ids:
            # Fetch the email using its ID
            status, msg_data = mail.fetch(recent_email_id, '(RFC822)')

            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    # Parse the email content
                    msg = email.message_from_bytes(response_part[1])

                    # Decode the subject properly
                    raw_subject = msg.get('Subject', '')
                    decoded_subject, encoding = decode_header(raw_subject)[0]
                    if isinstance(decoded_subject, bytes):
                        SUBJECT = decoded_subject.decode(encoding if encoding else 'utf-8')
                    else:
                        SUBJECT = decoded_subject

                    sender = msg.get('from', '')
                    TIMESTAMP = msg.get('date')

                    # Initialize variables for new fields
                    INCIDENT_ID, IMPACT_LEVEL, AFFECTED_SERVICE, DESCRIPTION = (None,) * 4

                    # Extract the body of the email
                    if msg.is_multipart():
                        body = ""
                        for part in msg.walk():
                            if part.get_content_type() in ['text/plain', 'text/html']:
                                body = part.get_payload(decode=True).decode('utf-8', errors='ignore')
                                break
                    else:
                        body = msg.get_payload(decode=True).decode('utf-8', errors='ignore')

                    # Extract specific details from the body
                    if "Incident ID:" in body:
                        INCIDENT_ID = body.split("Incident ID:")[1].splitlines()[0].strip()
                    if "Severity:" in body:
                        IMPACT_LEVEL = body.split("Severity:")[1].splitlines()[0].strip()
                    if "Affected System:" in body:
                        AFFECTED_SERVICE = body.split("Affected System:")[1].splitlines()[0].strip()

                    # Extract description (text between 'Dear Team,' and 'Incident Details:')
                    if "Dear Team," in body and "Incident Details:" in body:
                        start_index = body.find("Dear Team,") + len("Dear Team,")
                        end_index = body.find("Incident Details:")
                        DESCRIPTION = body[start_index:end_index].strip()

                    # Insert extracted details into Snowflake
                    try:
                        cur.execute("""
                            INSERT INTO email_alertdata (INCIDENT_ID, SUBJECT, IMPACT_LEVEL, AFFECTED_SERVICE, TIMESTAMP, DESCRIPTION)
                            VALUES (%s, %s, %s, %s, %s, %s);
                        """, (INCIDENT_ID, SUBJECT, IMPACT_LEVEL, AFFECTED_SERVICE, TIMESTAMP, DESCRIPTION))

                        print(f"Stored: {SUBJECT} from {sender}")
                        success_count += 1
                    except Exception as e:
                        print(f"Error inserting data: {e}")

        if conn:
            conn.commit()
        print(f"Successfully stored {success_count}/{email_count} emails.")
    else:
        print(f"Successfully stored 0/0 emails.")

except snowflake.connector.errors.DatabaseError as e:
    print(f"Snowflake Database Error: {e}")
    if conn:
        conn.rollback()
except Exception as e:
    print(f"Error during email processing: {e}")
finally:
    # Close all connections
    try:
        if mail:
            mail.close()
            mail.logout()
            print("Gmail connection closed.")
    except Exception as e:
        print(f"Error closing Gmail connection: {e}")

    try:
        if cur:
            cur.close()
            print("Snowflake cursor closed.")
    except Exception as e:
        print(f"Error closing Snowflake cursor: {e}")

    try:
        if conn:
            conn.close()
            print("Snowflake connection closed.")
    except Exception as e:
        print(f"Error closing Snowflake connection: {e}")

print("All connections closed.")


Found 1 alert emails
Fetching 1 alert emails received today (02-Apr-2025)...
Stored: Fwd: Backup Alert: Scheduled Database Backup Failed from Ananya Mehta <ananyamehta.work@gmail.com>
Successfully stored 1/1 emails.
Gmail connection closed.
Snowflake cursor closed.
Snowflake connection closed.
All connections closed.
